In [6]:
import os
import io
import json
import pymongo
from pprint import pprint as pp
import csv
from collections import namedtuple
import time

import twitter
import urllib.parse

import logging
import time
from datetime import datetime

class IO_json(object):
    def __init__(self, filepath, filename, filesuffix='json'):
        self.filepath = filepath
        self.filename = filename
        self.filesuffix = filesuffix
        
    def save(self, data):
        if os.path.isfile('{0}/{1}.{2}'.format(self.filepath, self.filename, self.filesuffix)):
            with io.open('{0}/{1}.{2}'.format(self.filepath, self.filename, self.filesuffix), 'a', encoding='utf-8') as f:
                f.write(json.dumps(data, ensure_ascii=False))
                
        else:
            with io.open('{0}/{1}.{2}'.format(self.filepath, self.filename, self.filesuffix), 'w', encoding='utf-8') as f:
                f.write(json.dumps(data, ensure_ascii=False))
                
    def load(self):
        with io.open('{0}/{1}.{2}'.format(self.filepath, self.filename, self.filesuffix), encoding='utf-8') as f:
            return f.read()
        

class IO_csv(object):
    def __init__(self, filepath, filename, filesuffix='csv'):
        self.filepath=filepath
        self.filename=filename
        self.filesuffix = filesuffix
        
    def save(self, data, NTname, fields):
        NTuple = namedtuple(NTname, fields)
        
        if os.path.isfile('{0}/{1}.{2}'.format(self.filepath, self.filename, self.filesuffix)):
            with open('{0}/{1}.{2}'.format(self.filepath, self.filename, self.filesuffix), 'ab') as f:
                writer = csv.writer(f)
                writer.writerows([row for row in map(NTuple._make,data)])
        else:
            with open('{0}/{1}.{2}'.format(self.filepath, self.filename, self.filesuffix), 'wb') as f:
                writer = csv.writer(f)
                writer.writerows(fields)
                writer.writerows([row for row in map(NTuple._make, data)])
                
    def load(self, NTname, fields):
        NTuple = namedtuple(NTName, fields)
        with open('{0}/{1}.{2}'.format(self.filepath, self.filename, self.filesuffix),'rU') as f:
            reader = csv.reader(f)
            for row in map(NTuple._make, reader):
                yield row
                
def parse_date(s):
    return time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(s,'%a %b %d %H:%M:%S +0000 %Y'))

def parse_geo(g,index):
    try:
        return str(g["geo"]["coordinates"][index])
    except:
        return ""
    
def extract_tweet(statuses):
    return [ {'id':status['id'], 
              'created_at':parse_date(status['created_at']), 
              'user_id':status['user']['id'], 
              'user_name':status['user']['name'],
              'tweet_text':status['text'].encode('utf-8'),
              'url':url['expanded_url']}
                    for status in statuses
                        for url in status['entities']['urls']
           ]

def extract_tweet_noURL(statuses):
    return [ {'id'         :status['id'], 
              'created_at' :parse_date(status['created_at']), 
              'user_id'    :status['user']['id'],
              'user_name'  :status['user']['name'], 
              'tweet_text' :status['text'].encode('utf-8') }
                               for status in statuses ]

fields01 = ['id','created_at','user_id','user_name','tweet_text','url']
Tweet01 = namedtuple('Tweet01', fields01)

def parse_tweet(data):
    return Tweet01(
        id=data.get('id',None),
        created_at=data.get('created_at', None),
        user_id=data.get('user_id', None),
        user_name=data.get('user_name', None),
        tweet_text=data.get('tweet_text', None),
        url=data.get('url')
    )

fields00 = ['id', 'created_at', 'user_id', 'user_name', 'tweet_text']
Tweet00 = namedtuple('Tweet00', fields00)

def parse_tweet_noURL(data):
    return Tweet00(
        id=data.get('id',None),
        created_at=data.get('created_at',None),
        user_id=data.get('user_id',None),
        user_name=data.get('user_name',None),
        tweet_text=data.get('tweet_text',None)
    )



class TwitterAPI(object):
    def __init__(self):
        consumer_key = 'bxZ2ypZgrxusqKvUGnlv1KNew'
        consumer_secret = 'HiAk5tuYc657MNKSb8odAxnU33fm4nLAAXY10PkhM3uUxbP9XB'
        access_token = '2983449373-pH31LSzmmrbgXrkZx08FWDnWqxNmWx74aMwLMt4'
        access_secret = 'Q41AgD2NdZ6n2UQWh8YM6nMosCKLENWrfqh1oiUNhp4R9'
        self.consumer_key = consumer_key
        self.consumer_secret = consumer_secret
        self.access_token = access_token
        self.access_secret = access_secret
        self.retries = 3
        self.auth = twitter.oauth.OAuth(access_token, access_secret, consumer_key, consumer_secret)
        self.api = twitter.Twitter(auth=self.auth)
        
        appName = 'twt150530'
        self.logger = logging.getLogger(appName)
        logPath = './'
        fileName = appName
        fileHandler = logging.FileHandler("{0}/{1}.log".format(logPath, fileName))
        formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
        fileHandler.setFormatter(formatter)
        self.logger.addHandler(fileHandler)
        self.logger.setLevel(logging.DEBUG)
        
        jsonFpath = './'
        jsonFname = 'twtr15053001'
        self.jsonSaver = IO_json(jsonFpath, jsonFname)
        
        #self.mongoSaver = IO_mongo(db='twtr01_db', coll='twtr01_coll')
    
    def searchTwitter(self, q, max_res=10, **kwargs):
        search_results = self.api.search.tweets(q=q, count=10, **kwargs)
        statuses = search_results['statuses']
        max_results = min(1000, max_res)
        
        for _ in range(10):
            try:
                next_results = search_results['search_metadata']['next_results']
            except KeyError as e:
                self.logger.error('error in searchTwitter: %s', e)
                break
                
            next_results = urllib.parse.parse_qsl(next_results[1:])
            kwargs = dict(next_results)
            search_results = self.api.search.tweets(**kwargs)
            statuses += search_results['statuses']
            self.saveTweets(search_results['statuses'])
                
            if len(statuses) > max_results:
                self.logger.info('info in searchTwitter - got %i tweets - max: %i' %(len(statuses), max_results))
                break
        
        return statuses
    
    def saveTweets(self, statuses):
        self.jsonSaver.save(statuses)
        
        #for s in statuses:
        #    self.mongoSaver.save(s)
            
    def parseTweets(self, statuses):
        return [ (status['id'],
                  status['created_at'],
                  status['user']['id'],
                  status['user']['name'],
                  status['text'],
                  url['expanded_url'])
                        for status in statuses
                            for url in status['entities']['urls']
                 ]
    
    def getTweets(self, q, max_res=10):
        def handleError(e, wait_period=2, sleep_when_rate_limited=True):
            if wait_period > 3600:
                self.logger.error('Too many retries in getTweets: %s', e)
                raise e
            if e.e.code == 401:
                self.logger.error('error 401 * Not Authorized * in getTweets: %s', e)
                return None
            elif e.e.code == 404:
                self.logger.error('error 404 * Not Found * in getTweets: %s', e)
                return None
            elif e.e.code == 429:
                self.logger.error('error 429 * API Rate Limit Exceeded * in getTweets: %s', e)
                if sleep_when_rate_limited:
                    self.logger.error('error 429 * Retrying in 15 minutes * in getTweets: %s', e)
                    sys.stderr.flush()
                    time.sleep(60*15 + 5)
                    self.logger.info('error 429 * Retrying now * in getTweets: %s', e)
                    return 2
                else:
                    raise e
            elif e.e.code in (500, 502, 503, 504):
                self.logger.info('Encountered %i Error. Retrying in %i seconds' % (e.e.code, wait_period))
                time.sleep(wait_period)
                wait_period *= 1.5
                return wait_period
            else:
                self.logger.error('Exit - aborting - %s', e)
                raise e
                
        while True:
            try:
                self.searchTwitter(q, max_res=10)
            except twitter.api.TwitterHTTPError as e:
                error_count=0
                wait_period = handleError(e, wait_period)
                if wait_period is None:
                    return
                

In [7]:
t = TwitterAPI()
q = 'ApacheSpark'
tsearch = t.searchTwitter(q)
#t.saveTweets(tsearch)
jsonFpath = './'
jsonFname = 'twtr15053001'

twts_ld = IO_json(jsonFpath, jsonFname).load()
twts_js = json.loads(twts_ld)

In [3]:
twts_ls_no_url = extract_tweet_noURL(twts_js)
pp(twts_ls_no_url)

[{'created_at': '2017-04-25 02:36:28',
  'id': 856698394487779330,
  'tweet_text': b'Running #BigDL, #DeepLearning for #ApacheSpark, on #AWS. #Bi'
                b'gData #DeepLearning #MachineLearning #DataScience #AI https:'
                b'//t.co/lf8GVEpvZw',
  'user_id': 146725639,
  'user_name': 'Sajeetharan'},
 {'created_at': '2017-04-25 01:26:56',
  'id': 856680896052600832,
  'tweet_text': b'RT @AnatolSokolenko: @ApacheSpark on @hadoop YARN #security '
                b'Model Analysis https://t.co/ZzGk60v9lO',
  'user_id': 2704548373,
  'user_name': 'NoSQL'},
 {'created_at': '2017-04-25 01:13:49',
  'id': 856677596108656640,
  'tweet_text': b"RT @Syncsort: .@ApacheSpark 2.0: A Look at What's New from @"
                b"Cloudera's @SeanAndersonBD https://t.co/WQA3WKPLdq #Cloudera"
                b' #spark link to @ho\xe2\x80\xa6',
  'user_id': 92932166,
  'user_name': 'Senthil Balakrishnan'},
 {'created_at': '2017-04-25 01:00:04',
  'id': 856674136290189312,
  'tweet_text':

In [8]:
twts_ls_url = extract_tweet(twts_js)
pp(twts_ls_url)

[{'created_at': '2017-04-25 02:36:28',
  'id': 856698394487779330,
  'tweet_text': b'Running #BigDL, #DeepLearning for #ApacheSpark, on #AWS. #Bi'
                b'gData #DeepLearning #MachineLearning #DataScience #AI https:'
                b'//t.co/lf8GVEpvZw',
  'url': 'https://aws.amazon.com/blogs/ai/?utm_content=buffer01731&utm_medium=social&utm_source=twitter.com&utm_campaign=buffer',
  'user_id': 146725639,
  'user_name': 'Sajeetharan'},
 {'created_at': '2017-04-25 01:26:56',
  'id': 856680896052600832,
  'tweet_text': b'RT @AnatolSokolenko: @ApacheSpark on @hadoop YARN #security '
                b'Model Analysis https://t.co/ZzGk60v9lO',
  'url': 'http://engineering.rallyhealth.com/bigdata/security/2017/04/15/apache-spark-on-yarn-security-model-analysis.html',
  'user_id': 2704548373,
  'user_name': 'NoSQL'},
 {'created_at': '2017-04-25 01:13:49',
  'id': 856677596108656640,
  'tweet_text': b"RT @Syncsort: .@ApacheSpark 2.0: A Look at What's New from @"
                b"Clou

In [10]:
twts_nt_no_url = [parse_tweet_noURL(t) for t in twts_ls_no_url]
print(twts_nt_no_url[1])

Tweet00(id=856680896052600832, created_at='2017-04-25 01:26:56', user_id=2704548373, user_name='NoSQL', tweet_text=b'RT @AnatolSokolenko: @ApacheSpark on @hadoop YARN #security Model Analysis https://t.co/ZzGk60v9lO')


In [11]:
twts_nt_url = [parse_tweet(t) for t in twts_ls_url]
print(twts_nt_url[1])

Tweet01(id=856680896052600832, created_at='2017-04-25 01:26:56', user_id=2704548373, user_name='NoSQL', tweet_text=b'RT @AnatolSokolenko: @ApacheSpark on @hadoop YARN #security Model Analysis https://t.co/ZzGk60v9lO', url='http://engineering.rallyhealth.com/bigdata/security/2017/04/15/apache-spark-on-yarn-security-model-analysis.html')


In [12]:
csvFpath = './'
csvFname = 'twtr15051401'
csvSuffix = 'csv'

twts_csv = IO_csv(csvFpath, csvFname, csvSuffix)
fields = ['id', 'created_at', 'user_id', 'user_name', 'tweet_text', 'url']
Tweet_NT = 'Tweet01'

twts_csv.save(twts_nt_url, Tweet_NT, fields)


TypeError: a bytes-like object is required, not 'str'